# XGBOOST Tuning WARNING TO BE DONE ON A VALIDATION SET

In [90]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

In [92]:
preprocessed_train_set = pd.read_csv('data/preprocessed_train_set.csv', sep=';')
train_target = pd.read_csv('data/train_target.csv', sep=';')

preprocessed_test_set = pd.read_csv('data/preprocessed_test_set.csv', sep=';')
test_target = pd.read_csv('data/test_target.csv', sep=';')

In [93]:
ids = ['application_id','customer_id']
train_ids = preprocessed_train_set[ids]
test_ids = preprocessed_test_set[ids]

preprocessed_train_set = preprocessed_train_set.drop(ids, 1)
preprocessed_test_set = preprocessed_test_set.drop(ids, 1)

In [94]:
booster =  XGBClassifier(
     eta = 0.01,
    max_depth=1,
    base_score=0.5,
    subsample=0.8, 
        colsample_bytree=0.8,
        gamma=1, 
        min_child_weight= 12,
        n_estimators=319,
        n_jobs=-1,
        nthread=None,
        objective = 'binary:logistic',
        eval_metric ='auc')    

In [95]:
booster.fit(preprocessed_train_set, np.ravel(train_target))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eta=0.01, eval_metric='auc', gamma=1,
       learning_rate=0.1, max_delta_step=0, max_depth=1,
       min_child_weight=12, missing=None, n_estimators=319, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [96]:
pred_labels = booster.predict(preprocessed_test_set)

In [97]:
roc_auc =  roc_auc_score(test_target, np.ravel(pred_labels))

In [98]:
roc_auc

0.6560186290900407

# Find Best Iteration

In [119]:
par = {  
    "eta": 0.01,
    "max_depth":15,
    "min_child_weight":12,
    "base_score":0.5,
    "n_estimators":5000,
    "n_jobs":-1,
    "subsample": 0.8,
    "colsample_bytree": 1,
    "gamma": 0,
    "objective" : 'binary:logistic',
    "eval_metric" :'auc'}

In [120]:
X_train, X_validation, y_train, y_validation =  train_test_split(
        preprocessed_train_set,
        train_target,
        test_size=.25,
        stratify=train_target
    )

In [121]:
 # Translate these sets into XGBoost-compatible dense matrices
d_train = xgb.DMatrix(X_train.values, label=y_train.values, feature_names=X_train.columns)
d_validation = xgb.DMatrix(X_validation, label=y_validation.values, feature_names=X_validation.columns)

# Parametrize a watch list to follow the training performance
watch_list = [(d_train, 'train'), (d_validation, 'validation')]



In [122]:
first_model = xgb.train(
    params=par,
    dtrain=d_train,
    num_boost_round=6000,
    evals=watch_list,
    early_stopping_rounds=100
    )


[0]	train-auc:0.642115	validation-auc:0.604053
Multiple eval metrics have been passed: 'validation-auc' will be used for early stopping.

Will train until validation-auc hasn't improved in 100 rounds.
[1]	train-auc:0.694797	validation-auc:0.658569
[2]	train-auc:0.712751	validation-auc:0.663548
[3]	train-auc:0.717131	validation-auc:0.687875
[4]	train-auc:0.717981	validation-auc:0.686851
[5]	train-auc:0.72104	validation-auc:0.691371
[6]	train-auc:0.727014	validation-auc:0.692147
[7]	train-auc:0.732042	validation-auc:0.687275
[8]	train-auc:0.734424	validation-auc:0.690559
[9]	train-auc:0.739559	validation-auc:0.681943
[10]	train-auc:0.742649	validation-auc:0.683956
[11]	train-auc:0.752572	validation-auc:0.683426
[12]	train-auc:0.751994	validation-auc:0.679472
[13]	train-auc:0.754269	validation-auc:0.681061
[14]	train-auc:0.756856	validation-auc:0.682579
[15]	train-auc:0.757584	validation-auc:0.677424
[16]	train-auc:0.75669	validation-auc:0.680107
[17]	train-auc:0.758478	validation-auc:0.6

In [123]:
# Should i use validation or train?

# Find Max Depth and Min Child Weight

In [65]:
   "eta": 0.01,
    "max_depth":3,
    "base_score":0.5,
    "n_estimators":5000,
    "n_jobs":-1,
    "subsample": 0.8,
    "colsample_bytree": 1,
    "gamma": 1,
    "objective" : 'binary:logistic',
    "eval_metric" :'auc'}

SyntaxError: invalid syntax (<ipython-input-65-155dd931e5b2>, line 1)

In [129]:
param_test1 = {
 'max_depth':[8,9,10,11,12],
 'min_child_weight':[7,8,9,10]
}

gsearch1 = GridSearchCV(
    estimator = XGBClassifier(
        learning_rate =0.01, 
        max_depth=3,
        subsample=0.8, 
        colsample_bytree=1,
        gamma=1, 
        objective= 'binary:logistic', 
        n_jobs=-1, 
        seed=2019,
        n_estimators=128),
    
    param_grid = param_test1, 
    scoring='roc_auc',
    n_jobs=-1,
    iid=False, 
    cv=3)

gsearch1.fit(preprocessed_train_set, train_target)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=128,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=2019, silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'max_depth': [8, 9, 10, 11, 12], 'min_child_weight': [7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [130]:
print(gsearch1.best_params_)
print(gsearch1.best_score_)

{'max_depth': 9, 'min_child_weight': 8}
0.6403695115515609


# Tune Gamma

In [126]:
[i/10.0 for i in range(0,5)]

[0.0, 0.1, 0.2, 0.3, 0.4]

In [131]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(
    estimator = XGBClassifier(
        learning_rate =0.01, 
        max_depth=9,
        subsample=0.8, 
        colsample_bytree=1,
        gamma=1, 
        min_child_weight= 8,
        objective= 'binary:logistic', 
        n_jobs=-1, 
        seed=2019,
        n_estimators=128),
    
    param_grid = param_test2, 
    scoring='roc_auc',
    n_jobs=-1,
    iid=False, 
    cv=5)

gsearch2.fit(preprocessed_train_set, train_target)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=9, min_child_weight=8, missing=None, n_estimators=128,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=2019, silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [132]:
print(gsearch2.best_params_)
print(gsearch2.best_score_)

{'gamma': 0.3}
0.6380649496080627


# Subsamble and  colsample_bytree

In [133]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch3 = GridSearchCV(
    estimator = XGBClassifier(
        learning_rate =0.01, 
        max_depth=9,
        subsample=0.8, 
        colsample_bytree=1,
        gamma=0.3, 
        min_child_weight= 8,
        objective= 'binary:logistic', 
        n_jobs=-1, 
        seed=2019,
        n_estimators=128),
    
    param_grid = param_test3, 
    scoring='roc_auc',
    n_jobs=-1,
    iid=False, 
    cv=5)

gsearch3.fit(preprocessed_train_set, train_target)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.3, learning_rate=0.01, max_delta_step=0,
       max_depth=9, min_child_weight=8, missing=None, n_estimators=128,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=2019, silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [134]:
print(gsearch3.best_params_)
print(gsearch3.best_score_)

{'subsample': 0.7, 'colsample_bytree': 0.7}
0.6493873990687806
